# Playlist Queries & Testing

This notebook served as a quick testing ground for playlist retrieval from the SpotifyAPI and some basic recommendation before creating the **Combined recommendations** notebook.
As a simple testing ground it is not really commented.

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from pathlib import Path
import pandas as pd
from spotifuncs import authenticate,create_df_top_songs, append_audio_features,create_similarity_score,get_recommendations, top_artists_from_API, create_df_saved_songs

In [2]:
path = Path("C:/Users/ms101/OneDrive/DataScience_ML/projects/spotify_app")

In [3]:
with open(path / "client_s.txt") as f:
    content = f.readlines()
content = [x.strip() for x in content]

client_id = content[0]
client_secret = content[1]

In [4]:
with open(path / "usernames.txt") as f:
    usernames = f.readlines()
usernames = [x.strip() for x in usernames]

username1 = usernames[0]
username2 = usernames[1]

In [5]:
scope = "user-library-read user-read-recently-played user-top-read playlist-modify-public playlist-read-private playlist-read-collaborative"

In [6]:
redirect_uri = "https://developer.spotify.com/dashboard/applications/4a4e029d299a4241873db8300038bf0a"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
                                                      client_secret=client_secret)

In [7]:
# def authenticate(redirect_uri, client_cred_manager, username, scope,client_id,client_secret):
#     sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
#     token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
#     if token:
#         sp = spotipy.Spotify(auth=token)
#     else:
#         print("Can't get token for", username)
#     return sp

In [8]:
sp = authenticate(redirect_uri, client_credentials_manager, username1, scope, client_id, client_secret)

In [6]:
# redirect_uri = "https://developer.spotify.com/dashboard/applications/4a4e029d299a4241873db8300038bf0a"


# client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
#                                                       client_secret=client_secret)

# sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


# token = util.prompt_for_user_token(username1, scope, client_id, client_secret, redirect_uri)

# if token:
#     sp = spotipy.Spotify(auth=token)
# else:
#     print("Can't get token for", username1)

In [9]:
##queries
#user top tracks
top_tracks_short = sp.current_user_top_tracks(limit = 50,offset=0,time_range='short_term')
top_tracks_med = sp.current_user_top_tracks(limit = 50,offset=0,time_range='medium_term')
top_tracks_long = sp.current_user_top_tracks(limit = 50,offset=0,time_range='long_term')
#user top artists
top_artists_long = sp.current_user_top_artists(limit = 50, time_range = "long_term")
top_artists_med = sp.current_user_top_artists(limit = 50, time_range = "medium_term")
top_artists_short = sp.current_user_top_artists(limit = 50, time_range = "short_term")
#user recent tracks
#still in BETA phase no spotipy function (workaround??)

#user saved tracks
user_saved_tracks = sp.current_user_saved_tracks(limit = 50)


In [10]:
# def top_artists_from_API(api_results):
#     df = pd.DataFrame(api_results["items"])
#     cols = ["name","id","genres","popularity","uri"]
#     return df[cols]

In [11]:
artists_short_df = top_artists_from_API(top_artists_short)
artists_med_df = top_artists_from_API(top_artists_med)
artists_long_df = top_artists_from_API(top_artists_long)
artists_df = pd.concat([artists_short_df,artists_med_df,artists_long_df])
artists_df["genres"] = artists_df["genres"].apply(lambda x: ",".join(x))
artists_df.drop_duplicates().reset_index(drop = True)
artists_df

,name,id,genres,popularity,uri
0,Joyner Lucas,6C1ohJrd5VydigQtaGy5Wa,"boston hip hop,hip hop,pop rap,rap",78,spotify:artist:6C1ohJrd5VydigQtaGy5Wa
1,Fynn Kliemann,2YIxQoGvBukvoC1CbJ7opS,german pop,65,spotify:artist:2YIxQoGvBukvoC1CbJ7opS
2,Geegun,5W7N6u4EjCEMKj7bDyzPEC,"russian dance,russian dance pop,russian hip ho...",57,spotify:artist:5W7N6u4EjCEMKj7bDyzPEC
3,Gidge,7aibxFH8hIlSUlXgshCgAP,"electronica,italian electronica,microhouse",49,spotify:artist:7aibxFH8hIlSUlXgshCgAP
4,Peter Fox,6rqlONGmPuP2wJVSfliLBI,"german hip hop,german pop,german rock",61,spotify:artist:6rqlONGmPuP2wJVSfliLBI
...,...,...,...,...,...
45,Chris Rock,36eSjIksD6fehqxyDUHDA3,"black comedy,comedy",48,spotify:artist:36eSjIksD6fehqxyDUHDA3
46,Carnage,7CCjtD0hCK005Bvg2WG1a7,"edm,electro house,electronic trap,pop rap,rap,...",64,spotify:artist:7CCjtD0hCK005Bvg2WG1a7
47,Wardruna,0NJ6wlOAsAJ1PN4VRdTPKA,"medieval folk,nordic folk,rune folk,viking folk",62,spotify:artist:0NJ6wlOAsAJ1PN4VRdTPKA
48,Motörhead,1DFr97A9HnbV3SKTJFu62M,"album rock,alternative metal,hard rock,metal,r...",68,spotify:artist:1DFr97A9HnbV3SKTJFu62M


Working with the saved tracks (only the last 50)

The dictionary follows a different structure than the one for top_tracks, a new function will be necessary. 

Let's first explore the structure and try to access the info we want.

In [10]:
user_saved_tracks["items"][0]["track"].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [11]:
user_saved_tracks["items"][0]["track"]["name"]

'I Fell In Love'

In [12]:
user_saved_tracks["items"][0]["track"]["album"]["name"]

'Autumn Bells'

In [13]:
user_saved_tracks["items"][0]["track"]["artists"][0]["name"]

'Gidge'

In [14]:
user_saved_tracks["items"][0]["track"]["popularity"]

44

In [15]:
user_saved_tracks["items"][0]["track"]["id"]

'0zrHx4EhLePXUa8KhGpS3E'

In [16]:
user_saved_tracks["items"][0]["track"]["duration_ms"]

385726

In [17]:
# def create_df_saved_songs(api_results):
#     track_name = []
#     track_id = []
#     artist = []
#     album = []
#     duration = []
#     popularity = []
#     for items in api_results["items"]:
#         try:
#             track_name.append(items["track"]['name'])
#             track_id.append(items["track"]['id'])
#             artist.append(items["track"]["artists"][0]["name"])
#             duration.append(items["track"]["duration_ms"])
#             album.append(items["track"]["album"]["name"])
#             popularity.append(items["track"]["popularity"])
#         except TypeError: 
#             pass

#     # Create the final df   
#     df = pd.DataFrame({ "track_name": track_name, 
#                              "album": album, 
#                              "track_id": track_id,
#                              "artist": artist, 
#                              "duration": duration, 
#                              "popularity": popularity})
#     return df

In [18]:
saved_tracks_df = create_df_saved_songs(user_saved_tracks)

In [19]:
saved_tracks_df

,track_name,album,track_id,artist,duration,popularity
0,I Fell In Love,Autumn Bells,0zrHx4EhLePXUa8KhGpS3E,Gidge,385726,44
1,The Beginning,The Beginning,4njhWDhTAjhReWtYkiMH9t,NR:TN,201904,44
2,Between Breaths,Stateless,4DJGTc1OsgqFsUGI6W8Mtx,Riyoon,479885,43
3,Berlin Nights,Berlin Nights,6gG1R1bFdJeNc2ERAwXxCb,Vnce Dolanbay,292115,39
4,Fall Slowly (feat. Ashanti) - Extended Version,Evolution,7wK4pOTZKVaAJ00rziu901,Joyner Lucas,292056,59
5,Evolution,Evolution,2VopDw2GlF3uwD1kihHmTT,Joyner Lucas,153250,62
6,Zim Zimma,Evolution,17nPeSliosCi427f0lUb75,Joyner Lucas,239702,70
7,Money In The Grave (Drake ft. Rick Ross),The Best In The World Pack,5ry2OE6R2zPQFDO85XkgRb,Drake,205426,81
8,Beifahrersitz,Beifahrersitz,01qOl2pM8emx1sxdBQc05g,LEA,199586,71
9,Twingo,POP,6vFEvkXOnOTUMgPHxogIRK,Fynn Kliemann,186984,56


In [20]:
#combine the top_tracks
top_tracks_short_df = append_audio_features(create_df_top_songs(top_tracks_short),sp)
top_tracks_med_df = append_audio_features(create_df_top_songs(top_tracks_med),sp)
top_tracks_long_df = append_audio_features(create_df_top_songs(top_tracks_long),sp)
top_tracks_df = pd.concat([top_tracks_short_df,top_tracks_med_df,top_tracks_long_df]).drop_duplicates().reset_index(drop = True)
top_tracks_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,10 Feet Down,Perception,68biLwi894rMQPeIiSky2t,NF,217106,63,0.620,0.835,5,-6.634,0,0.0984,0.096400,0.000012,0.269,0.0535,96.090
1,Let You Down,Perception,52okn5MNA47tk87PeZJLEL,NF,212120,82,0.662,0.714,5,-5.680,0,0.1210,0.312000,0.000000,0.179,0.4640,147.997
2,Fall Slowly (feat. Ashanti) - Extended Version,Evolution,7wK4pOTZKVaAJ00rziu901,Joyner Lucas,292056,59,0.542,0.453,6,-11.208,0,0.2400,0.204000,0.000007,0.137,0.0375,88.310
3,Zuhause,Nie,5vC4M4GjYLkgDaUwQcL7WA,Fynn Kliemann,197887,65,0.656,0.448,9,-8.676,0,0.0442,0.860000,0.000005,0.103,0.4030,95.626
4,Rolls Royce,Rolls Royce,22XFe65IH0P2RY0uowBqdI,Geegun,143437,68,0.872,0.423,1,-7.337,1,0.2130,0.142000,0.000010,0.104,0.3730,95.980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Like A G6,Like a G6,5S1yeuswEPofFOCkMFLNLc,Far East Movement,218346,0,0.629,0.869,7,-7.013,0,0.3140,0.006570,0.000000,0.191,0.7150,125.024
138,Halftime,Wolke 7,1N9ZAbqVw5o0m7wccSgRIt,Gzuz,181060,1,0.533,0.783,2,-6.490,1,0.3250,0.181000,0.000000,0.101,0.4710,129.844
139,Push It,Wisconsin Death Trip,43WFwjiWFHc8ZryT1Tz1aY,Static-X,154906,36,0.557,0.977,0,-3.965,1,0.0493,0.000393,0.000004,0.291,0.5040,149.879
140,"Talk Shit, Get Shot",Manslaughter,2rsho1vI3L5o65QOjM5Mwx,Body Count,227147,39,0.455,0.993,3,-1.952,0,0.2930,0.002350,0.000000,0.149,0.3840,175.004


In [21]:
saved_tracks_df = append_audio_features(saved_tracks_df,sp)

In [22]:
saved_tracks_df.head(10)

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,I Fell In Love,Autumn Bells,0zrHx4EhLePXUa8KhGpS3E,Gidge,385726,44,0.539,0.610,8,-10.999,0,0.1340,0.12100,0.809000,0.0857,0.0395,123.218
1,The Beginning,The Beginning,4njhWDhTAjhReWtYkiMH9t,NR:TN,201904,44,0.783,0.714,7,-7.408,1,0.2460,0.24300,0.860000,0.4460,0.0341,125.955
2,Between Breaths,Stateless,4DJGTc1OsgqFsUGI6W8Mtx,Riyoon,479885,43,0.738,0.665,7,-10.441,0,0.0438,0.00748,0.877000,0.5820,0.0347,99.992
3,Berlin Nights,Berlin Nights,6gG1R1bFdJeNc2ERAwXxCb,Vnce Dolanbay,292115,39,0.901,0.457,10,-13.238,0,0.1630,0.22900,0.424000,0.0977,0.5310,127.999
4,Fall Slowly (feat. Ashanti) - Extended Version,Evolution,7wK4pOTZKVaAJ00rziu901,Joyner Lucas,292056,59,0.542,0.453,6,-11.208,0,0.2400,0.20400,0.000007,0.1370,0.0375,88.310
5,Evolution,Evolution,2VopDw2GlF3uwD1kihHmTT,Joyner Lucas,153250,62,0.687,0.819,9,-6.670,0,0.4310,0.21800,0.000000,0.3920,0.5680,81.185
6,Zim Zimma,Evolution,17nPeSliosCi427f0lUb75,Joyner Lucas,239702,70,0.883,0.621,11,-6.063,0,0.2120,0.08710,0.000000,0.4990,0.6760,149.052
7,Money In The Grave (Drake ft. Rick Ross),The Best In The World Pack,5ry2OE6R2zPQFDO85XkgRb,Drake,205426,81,0.831,0.502,10,-4.045,0,0.0460,0.10100,0.000000,0.1220,0.1010,100.541
8,Beifahrersitz,Beifahrersitz,01qOl2pM8emx1sxdBQc05g,LEA,199586,71,0.712,0.774,6,-3.967,0,0.1330,0.34600,0.000000,0.1760,0.4710,159.977
9,Twingo,POP,6vFEvkXOnOTUMgPHxogIRK,Fynn Kliemann,186984,56,0.619,0.638,2,-5.754,0,0.0906,0.29700,0.000001,0.4400,0.3600,141.967


In [23]:
similarity_top_tracks = create_similarity_score(top_tracks_df,top_tracks_df)
similarity_saved_tracks = create_similarity_score(saved_tracks_df,saved_tracks_df)

In [24]:
for track in top_tracks_df["track_name"].sample(5):
    print(track, "\n")
    print(get_recommendations(top_tracks_df,track,similarity_top_tracks))
    print("\n")

MOSKAU 

112        Light Rain
27            Crainte
21     Slap the Ghost
89          Magnifico
22               Zver
Name: track_name, dtype: object


FEUER 

22                          Zver
128       I Mean It (feat. Remo)
4                    Rolls Royce
51                          YAYA
16     Creep - R3HAB Chill Remix
Name: track_name, dtype: object


Nero Forte 

75                                              Holy War
60                                              Unspoken
116                                The Sound of Violence
95     Still A Full Gentle Racket - Angerfist 2019 Refix
103                                           Gold Cobra
Name: track_name, dtype: object


m.A.A.d city 

112       Light Rain
88       Holy Roller
97      m.A.A.d city
107    6 Foot 7 Foot
75          Holy War
Name: track_name, dtype: object


Huldra 

47                                                Huldra
49                                Huldra - Other Version
24     In The (Last) Moment - R

In [25]:
for track in saved_tracks_df["track_name"].sample(5):
    print(track, "\n")
    print(get_recommendations(saved_tracks_df,track,similarity_saved_tracks))
    print("\n")

Idol (feat. Tech N9ne) 

24        Slap the Ghost
41                Omertà
34              Over Now
27    Lost In The Static
38               BURNING
Name: track_name, dtype: object


Light Rain 

26         Full Metal
2     Between Breaths
38            BURNING
29             So Far
0      I Fell In Love
Name: track_name, dtype: object


I Fell In Love 

0                                     I Fell In Love
33                                        Light Rain
4     Fall Slowly (feat. Ashanti) - Extended Version
2                                    Between Breaths
14                    Risky Business - Mathame Remix
Name: track_name, dtype: object


When I'm Small 

32                 YAYA
25    You.Won't.Do.Shit
16             Blackout
10                 ADHD
39                Alarm
Name: track_name, dtype: object


Generation YouPorn - Agia Napa Edit 

29                                            So Far
21                                      Five Minutes
4     Fall Slowly (feat. Ash

In [26]:
##Playlists
#spotify features playlists + get playlist by id
#featured_playlists, playlist

For later use:

current_user_saved_tracks
- Gets a list of the tracks saved in the current authorized user’s “Your Music” library

artist_top_tracks
- Get Spotify catalog information about an artist’s top 10 tracks by country. 

artist_related_artists
- Get Spotify catalog information about artists similar to an identified artist. Similarity is based on analysis of the Spotify community’s listening history.

playlist_add_items

recommendations(seed_artists=None, seed_genres=None, seed_tracks=None, limit=20, country=None, \*\*kwargs)
- Get a list of recommended tracks for one to five seeds. (at least one of seed_artists, seed_tracks and seed_genres are needed)

- Parameters:
- seed_artists - a list of artist IDs, URIs or URLs
- seed_tracks - a list of track IDs, URIs or URLs
- seed_genres - a list of genre names. Available genres for
- recommendations can be found by calling recommendation_genre_seeds

user_playlist_replace_tracks(user, playlist_id, tracks)
